<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC2_FD004_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
%pip install scikeras;
%pip install -U tensorflow-addons;
%pip install scikit-optimize

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import datetime

H:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
H:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
H:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import GroupKFold

from sklearn.base import BaseEstimator,RegressorMixin

In [4]:
from skopt import BayesSearchCV

from skopt.space.space import Categorical, Integer, Real

In [5]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Masking

from tensorflow.keras.metrics import RootMeanSquaredError as RMSE

from keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay

import tensorflow_addons as tfa
from tensorflow_addons.metrics import RSquare as R2

from scikeras.wrappers import KerasRegressor

In [6]:
sns.set_palette('colorblind')

In [7]:
# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [8]:
tf.config.experimental.enable_op_determinism()

In [9]:
# Remove some tf warnings
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

# Data Preparation

In [10]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    IN_COLAB = True
    folder="/content/"

    # Dataset Download 
    os.system('git clone https://github.com/arthursl12/dataset_2')
    os.system('mv /content/dataset_2/CMaps /content/CMaps')
    os.system('mv /content/dataset_2/data_processing /content/data_processing')
    os.system('rm -rf dataset_2')
else:
    print('Not running on CoLab')
    IN_COLAB = False
    folder="CMaps/"
    %cd dataset_2/

Not running on CoLab
C:\Users\Arthur Lima\POC\dataset_2


In [11]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [12]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [13]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(4, folder='CMaps/')
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61244,249,251,9.9998,0.2500,100.0,489.05,605.33,1516.36,1315.28,10.52,...,380.16,2388.73,8185.69,8.4541,0.03,372,2319,100.00,29.11,17.5234
61245,249,252,0.0028,0.0015,100.0,518.67,643.42,1598.92,1426.77,14.62,...,535.02,2388.46,8185.47,8.2221,0.03,396,2388,100.00,39.38,23.7151
61246,249,253,0.0029,0.0000,100.0,518.67,643.68,1607.72,1430.56,14.62,...,535.41,2388.48,8193.94,8.2525,0.03,395,2388,100.00,39.78,23.8270
61247,249,254,35.0046,0.8400,100.0,449.44,555.77,1381.29,1148.18,5.48,...,187.92,2388.83,8125.64,9.0515,0.02,337,2223,100.00,15.26,9.0774


## Preprocessing

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [14]:
test.shape, y_test.shape

((41214, 26), (248, 1))

In [15]:
test_last = proc.transform_test_keep_setting(test)
test_last.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,25.0070,0.6214,60.0,462.54,537.66,1264.31,1046.41,7.05,8.99,176.56,...,166.19,2028.53,7890.31,10.7615,0.02,308,1915,84.93,14.41,8.6329
1,41.9989,0.8400,100.0,445.00,549.96,1354.05,1133.55,3.91,5.72,139.03,...,130.17,2387.72,8073.44,9.3925,0.02,331,2212,100.00,10.58,6.4325
2,42.0005,0.8401,100.0,445.00,549.47,1341.06,1118.90,3.91,5.69,139.26,...,130.73,2388.18,8095.58,9.2974,0.02,330,2212,100.00,10.61,6.3488
3,25.0018,0.6207,60.0,462.54,536.06,1253.49,1038.53,7.05,9.00,175.63,...,164.91,2028.30,7878.63,10.8396,0.02,306,1915,84.93,14.41,8.5696
4,25.0039,0.6200,60.0,462.54,537.36,1263.60,1052.52,7.05,9.03,175.53,...,164.95,2028.24,7873.75,10.9094,0.02,307,1915,84.93,14.19,8.6248


In [16]:
X_test = test_last

### Remaining Useful Life (RUL)

In [17]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,320
1,1,2,319
2,1,3,318
3,1,4,317
4,1,5,316


## Attributes and target separation

In [18]:
X_train, y_train = proc.X_y_train_divide_with_settings(train)

In [19]:
y_train.head()

,RUL
0,320
1,319
2,318
3,317
4,316


In [20]:
X_train.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,...,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,...,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,...,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,5.70,137.98,...,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,9.00,174.82,...,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754


## Training and Evaluation functions

In [21]:
eval = Evaluation()

In [22]:
search = HyperparameterSearch()

# LSTM Construction

## Callbacks

In [23]:
# Early Stopping Callback
es = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                      patience=5, restore_best_weights=True)

In [24]:
# Printing Callback
def printLog(epoch, logs):
    print(
        f"E {epoch+1}\t: loss={logs['loss']:.3f}, "+
        f"rmse={logs['root_mean_squared_error']:.3f}, "+
        f"r2={logs['r_square']:.3f}; "+
        f"v_loss={logs['val_loss']:.3f}, "+
        f"v_rmse={logs['val_root_mean_squared_error']:.3f}, "+
        f"v_r2={logs['val_r_square']:.3f}; "
    )

printerCallback = LambdaCallback(on_epoch_end=printLog)

## Constants

In [25]:
# X_train must include indices
train3 = train.copy()
X_train_ = train3.drop(columns=["RUL"])

In [26]:
INPUT_SHAPE = 0

## Wrapper

In [27]:
from sklearn.metrics import r2_score

class LSTMWrapperRegressor(BaseEstimator,RegressorMixin):
    def __init__(self, basemodel=None, clip_y=-1, seq_length=40,
                 include_settings=False, poly_degree=1,
                 scaler=StandardScaler()):
        # Base parameters
        self.basemodel = basemodel
        self.clip_y = clip_y
        self.seq_length = seq_length
        self.poly_degree = poly_degree
        self.include_settings = include_settings

        # Column indexers
        self.feature_cols = sensors_cols
        if(include_settings):
            # self.seq_cols = settings_cols + self.cols
            self.feature_cols = settings_cols + self.feature_cols
        self.seq_cols = ["time"] + self.feature_cols
        self.base_feature_cols = self.feature_cols

        # Scaler and PolyFeatures transformers
        self.scaler = scaler
        self.polyft = PolynomialFeatures(degree=self.poly_degree, 
                                         include_bias=False)



    def fit(self, X=None, y=None):
        # Merge features and target again
        data = X.copy()
        data["RUL"] = y

        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.fit_transform(data[self.base_feature_cols])
        transf = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out(),
                              index=data.index)
        self.feature_cols = list(self.polyft.get_feature_names_out())
        self.seq_cols = ["time"] + self.feature_cols
        data[self.feature_cols] = transf

        # Scale the data
        data[self.feature_cols] = \
                            self.scaler.fit_transform(data[self.feature_cols])

        # Transform into time series
        X_train = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        # print(data.shape,X_train.shape, X.shape)

        # Clip and transform labels
        data2 = data.copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_train = self.gen_y_wrapper(data2,self.seq_length,["RUL"])

        # Update input shape for future use
        global INPUT_SHAPE
        # print(INPUT_SHAPE, X_train.shape)
        INPUT_SHAPE = (X_train.shape[1],X_train.shape[2])

        # Fit model
        # print(X_train.shape, y_train.shape)
        self.basemodel.fit(X_train,y_train)
        return self
        
    def predict(self, X=None):
        # Perform transformation, if not done
        if (len(X.shape) < 3):
            data = X.copy()

            # Apply polynomial features
            transf = self.polyft.transform(data[self.base_feature_cols])
            transf = pd.DataFrame(transf, 
                                  columns=self.polyft.get_feature_names_out(),
                                  index=data.index)
            # data = pd.concat([data,transf], axis=1)
            data[self.feature_cols] = transf

            # Scale the data
            data[self.feature_cols] = \
                            self.scaler.transform(data[self.feature_cols])
            
            # Transform into time series
            X_train = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        else:
            X_train = X
        return self.basemodel.predict(X_train)

    def score(self, X, y, sample_weight=None):
        # Merge features and target again
        data = X.copy()
        data["RUL"] = y

        # Apply polynomial features
        transf = self.polyft.transform(data[self.base_feature_cols])
        transf = pd.DataFrame(transf, 
                              columns=self.polyft.get_feature_names_out(),
                              index=data.index)
        # data = pd.concat([data,transf], axis=1)
        data[self.feature_cols] = transf

        # Scale the data (with train data parameters)
        data[self.feature_cols] = \
                        self.scaler.transform(data[self.feature_cols])
        
        # Transform into time series
        X_test = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        # print(data.shape,X_test.shape, X.shape)

        # Clip and transform labels
        data2 = data.copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_test = self.gen_y_wrapper(data2,self.seq_length,["RUL"])

        # Predict on test data
        y_pred = self.predict(X_test)
        return r2_score(y_test, y_pred, sample_weight=sample_weight)

    def gen_X_data(self, df, sequence_length, columns, mask_value=-99.):
        if df.shape[0] < sequence_length:
            # print("\t Not enough sequence:",df.shape[0]," < ",sequence_length)
            data = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
            idx = data.shape[0] - df.shape[0]
            data[idx:,:] = df[columns].values  # fill with available data
        else:
            data = df[columns].values
            
        # # specifically yield the last possible sequence
        # stop = num_elements = data_matrix.shape[0]
        # start = stop - sequence_length
        # for i in list(range(1)):
        #     yield data_matrix[start:stop, :]



        # data = df[columns].values
        num_elements = data.shape[0]

        # -1 and +1 because of Python indexing
        for start, stop in zip(range(0, num_elements-(sequence_length-1)), 
                               range(sequence_length, num_elements+1)):
            yield data[start:stop, :]

    def gen_X_wrapper(self, df, sequence_length, columns, 
                      unit_nrs=np.array([]), idx_col="unit_number"):
        # print(">> Wrapper called. df shape:",df.shape)
        if unit_nrs.size <= 0:
            unit_nrs = df[idx_col].unique()
            
        data_gen = (list(self.gen_X_data(df[df[idx_col]==unit_nr], 
                                         sequence_length, columns))
                for unit_nr in unit_nrs)
        # print("\tdatagen len:",len(data_gen))
        data_array = np.concatenate(list(data_gen)).astype(np.float32)
        # print("\tdata_array.shape:",data_array.shape)
        return data_array

    def gen_y(self, df, sequence_length, label):
        data_matrix = df[label].values
        num_elements = data_matrix.shape[0]

        # -1 because I want to predict the rul of that last row in the sequence, 
        # not the next row
        return data_matrix[sequence_length-1:num_elements, :]  

    def gen_y_wrapper(self, df, sequence_length, label, 
                      unit_nrs=np.array([]), idx_col="unit_number"):
        # print(">> Y Wrapper called. df shape:",df.shape)
        if unit_nrs.size <= 0:
            unit_nrs = df[idx_col].unique()
            
        label_gen = [self.gen_y(df[df[idx_col]==unit_nr], 
                                sequence_length, label) 
                    for unit_nr in unit_nrs]
        # print("\tlabelgen len:",len(label_gen))
        label_array = np.concatenate(label_gen).astype(np.float32)
        # print("\tlabel_array.shape:",label_array.shape)
        return label_array

## Test Data

In [28]:
def gen_test_data(df, sequence_length, columns, mask_value):
    if df.shape[0] < sequence_length:
        data_matrix = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
        idx = data_matrix.shape[0] - df.shape[0]
        data_matrix[idx:,:] = df[columns].values  # fill with available data
    else:
        data_matrix = df[columns].values
        
    # specifically yield the last possible sequence
    stop = num_elements = data_matrix.shape[0]
    start = stop - sequence_length
    for i in list(range(1)):
        yield data_matrix[start:stop, :]

In [29]:
def gen_test_wrapper(X_test_scaled, sequence_length, cols, idx_col="unit_number"): 
    data_gen = (
        list(gen_test_data(X_test_scaled[X_test_scaled[idx_col]==unit_nr], 
                           sequence_length, cols, -99.))
            for unit_nr in X_test_scaled[idx_col].unique())
    data_array = np.concatenate(list(data_gen)).astype(np.float32)
    return data_array

In [30]:
def scale_test(test,model):
    test2 = test.copy()

    # Apply polynomial features
    transf = model.polyft.transform(test2[model.base_feature_cols])
    transf = pd.DataFrame(transf, 
                          columns=model.polyft.get_feature_names_out(),
                          index=test2.index)
    newcols = model.polyft.get_feature_names_out()
    test2[newcols] = transf 

    # Scale the data (with train data parameters)
    test2[model.feature_cols] = \
                    model.scaler.transform(test2[model.feature_cols])
    return test2

## Constructor

In [31]:
def create_model(optim=Adam, learning_rate=1e-3, 
                 layer1=32  , activation1="tanh"    , dropout1=0.1,
                 layer2=None, activation2="tanh"    , dropout2=0.1,
                 layer3=None, activation3="tanh"    , dropout3=0.1,
                 second_dense=True,
                 print_summary=False, loss='mean_squared_error',
                 metrics=[tf.keras.metrics.MeanSquaredError()]):
    model = Sequential()

    # Input-masked layer
    model.add(Masking(mask_value=-99., input_shape=INPUT_SHAPE))
    
    if (layer2 is None and layer3 is None):
        # Single LSTM layer
        model.add(LSTM(layer1, activation=activation1))
        model.add(Dropout(dropout1))
    elif (layer2 is not None and layer3 is None):
        # LSTM-Dense
        model.add(LSTM(layer1, activation=activation1))
        model.add(Dropout(dropout1))
        model.add(Dense(layer2, activation=activation2))
        model.add(Dropout(dropout2))
    elif (layer2 is not None and layer3 is not None and second_dense==False):
        # LSTM-LSTM-Dense
        model.add(LSTM(layer1, activation=activation1, return_sequences=True))
        model.add(Dropout(dropout1))
        model.add(LSTM(layer2, activation=activation2))
        model.add(Dropout(dropout2))
        model.add(Dense(layer3, activation=activation3))
        model.add(Dropout(dropout3))
    elif (layer2 is not None and layer3 is not None and second_dense==True):
        # LSTM-Dense-Dense
        model.add(LSTM(layer1, activation=activation1))
        model.add(Dropout(dropout1))
        model.add(Dense(layer2, activation=activation2))
        model.add(Dropout(dropout2))
        model.add(Dense(layer3, activation=activation3))
        model.add(Dropout(dropout3))

    # Output Layer
    model.add(Dense(1))

    model.compile(loss=loss, optimizer=optim(learning_rate=learning_rate), 
                  metrics=metrics)
    
    if(print_summary): model.summary()
    return model

# LSTM 1-layer

## Linear RUL


Score: 0.565997282764165  
Test: 0.
```
('basemodel__batch_size', 354),
('basemodel__epochs', 43),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__dropout1', 0.3412271023238272),
('basemodel__model__layer1', 117),
('basemodel__model__learning_rate', 0.008518900361139942),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.17000957225461616),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 30)
```



In [32]:
SEQ_LENGTH=30
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=354,
                           epochs=43,
                           model__activation1='tanh',
                           model__dropout1=0.3412271023238272, 
                           model__layer1=117, 
                           model__learning_rate=0.008518900361139942,
                           model__optim=Adam,
                           validation_split=0.17000957225461616, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [33]:
model.fit(X_train_, y_train)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 30, 25)            0         
                                                                 
 lstm (LSTM)                 (None, 117)               66924     
                                                                 
 dropout (Dropout)           (None, 117)               0         
                                                                 
 dense (Dense)               (None, 1)                 118       
                                                                 
Total params: 67,042
Trainable params: 67,042
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=12649.581, rmse=112.470, r2=-0.767; v_loss=6483.583, v_rmse=80.521, v_r2=-0.077; 
E 2	: loss=7031.255, rmse=83.853, r2=0.018; v_loss=4797.660, v_rmse=69.265, v_r2=0.203; 
E 

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=354, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002007EAFDFD0>, <keras.callbacks.LambdaCallback object at 0x0000020009161910>], epochs=43, model=<function create_model at 0x0000020009692AF0>, model__activation1='tanh', model__dropout1=0.3412271023238272, model__layer1=117, model__learning_rate=0.008518900361139942, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x0000020008548F70>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000002000916C850>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.17000957225461616, verbose=0),
                     include_settings=True, seq_length=30)

In [34]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.365,RMSE=-43.447
Finished: 2022-11-03 09:56:07.897610


## Non-Linear RUL 

Score: 0.7343157247269209  
Test: 0.
```
('basemodel__batch_size', 32),
('basemodel__epochs', 41),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__dropout1', 0.23186170929942812),
('basemodel__model__layer1', 333),
('basemodel__model__learning_rate', 0.0016624190580454845),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.19268056108985515),
('clip_y', 134),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 30)
```


In [35]:
SEQ_LENGTH=30
CLIP=134

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=41,
                           model__activation1='tanh',
                           model__dropout1=0.23186170929942812, 
                           model__layer1=333, 
                           model__learning_rate=0.0016624190580454845,
                           model__optim=Adam,
                           validation_split=0.19268056108985515, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [36]:
model.fit(X_train_, y_train)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_1 (Masking)         (None, 30, 25)            0         
                                                                 
 lstm_1 (LSTM)               (None, 333)               478188    
                                                                 
 dropout_1 (Dropout)         (None, 333)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 334       
                                                                 
Total params: 478,522
Trainable params: 478,522
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=1823.237, rmse=42.699, r2=0.089; v_loss=1188.286, v_rmse=34.472, v_r2=0.409; 
E 2	: loss=1323.555, rmse=36.381, r2=0.339; v_loss=1166.059, v_rmse=34.148, v_r2=0.420; 
E 

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002007EAFDFD0>, <keras.callbacks.LambdaCallback object at 0x0000020009161910>], epochs=41, model=<function create_model at 0x0000020009692AF0>, model__activation1='tanh', model__dropout1=0.23186170929942812, model__layer1=333, model__learning_rate=0...624190580454845, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x0000020025125E50>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000002007EAFDE50>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.19268056108985515, verbose=0),
                     clip_y=134, include_settings=True, seq_length=30)

In [37]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.725,RMSE=-23.884
Finished: 2022-11-03 10:10:46.590054


# LSTM-Dense-1

## Linear RUL



Score: 0.5081060519759618  
Test: 0.
```
('basemodel__batch_size', 304),
('basemodel__epochs', 31),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'relu'),
('basemodel__model__dropout1', 0.3074381764174994),
('basemodel__model__dropout2', 0.5435400589103845),
('basemodel__model__layer1', 16),
('basemodel__model__layer2', 254),
('basemodel__model__learning_rate', 0.006871390161766782),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__model__second_dense', True),
('basemodel__validation_split', 0.2925386448085039),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 34)
```




In [38]:
SEQ_LENGTH=34
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=304,
                           epochs=31,
                           model__activation1='tanh',
                           model__activation2='relu',
                           model__dropout1=0.3074381764174994, 
                           model__dropout2=0.5435400589103845, 
                           model__layer1=16, 
                           model__layer2=254, 
                           model__learning_rate=0.006871390161766782,
                           model__optim=Adam,
                           model__second_dense=True,
                           validation_split=0.2925386448085039, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [39]:
model.fit(X_train_, y_train)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_2 (Masking)         (None, 34, 25)            0         
                                                                 
 lstm_2 (LSTM)               (None, 16)                2688      
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 254)               4318      
                                                                 
 dropout_3 (Dropout)         (None, 254)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 255       
                                                                 
Total params: 7,261
Trainable params: 7,261
Non-traina

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=304, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002007EAFDFD0>, <keras.callbacks.LambdaCallback object at 0x0000020009161910>], epochs=31, model=<function create_model at 0x0000020009692AF0>, model__activation1='tanh', model__activation2='relu', model__dropout1=0.3074381764174994, model__dropout...82, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000002001A7F8460>, <tensorflow_addons.metrics.r_square.RSquare object at 0x0000020008548040>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, model__second_dense=True, print_summary=True, validation_split=0.2925386448085039, verbose=0),
                     include_settings=True, seq_length=34)

In [40]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.504,RMSE=-38.404
Finished: 2022-11-03 10:14:38.695897


## Non-Linear RUL 

Score: 0.730501800938371  
Test: 0.
```
('basemodel__batch_size', 225),
('basemodel__epochs', 20),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'relu'),
('basemodel__model__dropout1', 0.3310559320251978),
('basemodel__model__dropout2', 0.3142813474813816),
('basemodel__model__layer1', 473),
('basemodel__model__layer2', 468),
('basemodel__model__learning_rate', 0.0008125349625551938),
('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
('basemodel__model__second_dense', True),
('basemodel__validation_split', 0.1),
('clip_y', 103),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 65)
```


In [59]:
SEQ_LENGTH=65
CLIP=103

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=225,
                           epochs=20,
                           model__activation1='tanh',
                           model__activation2='relu',
                           model__dropout1=0.3310559320251978, 
                           model__dropout2=0.3142813474813816, 
                           model__layer1=473, 
                           model__layer2=468, 
                           model__learning_rate=0.0008125349625551938,
                           model__optim=RMSprop,
                           model__second_dense=True,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [60]:
model.fit(X_train_, y_train)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_9 (Masking)         (None, 65, 25)            0         
                                                                 
 lstm_11 (LSTM)              (None, 473)               944108    
                                                                 
 dropout_20 (Dropout)        (None, 473)               0         
                                                                 
 dense_18 (Dense)            (None, 468)               221832    
                                                                 
 dropout_21 (Dropout)        (None, 468)               0         
                                                                 
 dense_19 (Dense)            (None, 1)                 469       
                                                                 
Total params: 1,166,409
Trainable params: 1,166,409
No

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=225, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002007EAFDFD0>, <keras.callbacks.LambdaCallback object at 0x0000020009161910>], epochs=20, model=<function create_model at 0x0000020009692AF0>, model__activation1='tanh', model__activation2='relu', model__dropout1=0.3310559320251978, model__dropout..., model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x0000020185485C40>, <tensorflow_addons.metrics.r_square.RSquare object at 0x0000020174E42580>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__second_dense=True, print_summary=True, validation_split=0.1, verbose=0),
                     clip_y=103, include_settings=True, seq_length=65)

In [62]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.538,RMSE=-24.343
Finished: 2022-11-03 12:33:35.991428


# LSTM-Dense-2 

## Linear RUL 


Score: 0.4975628097395217  
Test: 0.
```
('basemodel__batch_size', 183),
('basemodel__epochs', 31),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__activation3', 'elu'),
('basemodel__model__dropout1', 0.3224261325682778),
('basemodel__model__dropout2', 0.45581827067675496),
('basemodel__model__dropout3', 0.26574521774822357),
('basemodel__model__layer1', 512),
('basemodel__model__layer2', 339),
('basemodel__model__layer3', 50),
('basemodel__model__learning_rate', 0.0010131951305302183),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__model__second_dense', True),
('basemodel__validation_split', 0.8409177578463917),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 30)
```




In [47]:
SEQ_LENGTH=30
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=183,
                           epochs=31,
                           model__activation1='tanh',
                           model__activation2='elu',
                           model__activation3='elu',
                           model__dropout1=0.3224261325682778, 
                           model__dropout2=0.45581827067675496,
                           model__dropout3=0.26574521774822357, 
                           model__layer1=512, 
                           model__layer2=339, 
                           model__layer3=50, 
                           model__learning_rate=0.0010131951305302183,
                           model__optim=Adam,
                           model__second_dense=True,
                           validation_split=0.8409177578463917, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [48]:
model.fit(X_train_, y_train)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_5 (Masking)         (None, 30, 25)            0         
                                                                 
 lstm_5 (LSTM)               (None, 512)               1101824   
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 339)               173907    
                                                                 
 dropout_9 (Dropout)         (None, 339)               0         
                                                                 
 dense_9 (Dense)             (None, 50)                17000     
                                                                 
 dropout_10 (Dropout)        (None, 50)               

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=183, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002007EAFDFD0>, <keras.callbacks.LambdaCallback object at 0x0000020009161910>], epochs=31, model=<function create_model at 0x0000020009692AF0>, model__activation1='tanh', model__activation2='elu', model__activation3='elu', model__dropout1=0.3224261...83, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000002015CA53D60>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000201865DF610>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, model__second_dense=True, print_summary=True, validation_split=0.8409177578463917, verbose=0),
                     include_settings=True, seq_length=30)

In [49]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=-0.056,RMSE=-56.017
Finished: 2022-11-03 10:40:45.938988


## Non-Linear RUL 


Score: 0.7786741619787652  
Test: 0.
```
('basemodel__batch_size', 229),
('basemodel__epochs', 37),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'relu'),
('basemodel__model__activation3', 'sigmoid'),
('basemodel__model__dropout1', 0.4312949059884308),
('basemodel__model__dropout2', 0.3807450679193153),
('basemodel__model__dropout3', 0.6916033873523364),
('basemodel__model__layer1', 167),
('basemodel__model__layer2', 337),
('basemodel__model__layer3', 289),
('basemodel__model__learning_rate', 0.0013556548021189216),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__model__second_dense', True),
('basemodel__validation_split', 0.16231298330980526),
('clip_y', 120),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 33)
```



In [50]:
SEQ_LENGTH=33
CLIP=120

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=229,
                           epochs=37,
                           model__activation1='tanh',
                           model__activation2='relu',
                           model__activation3='sigmoid',
                           model__dropout1=0.4312949059884308, 
                           model__dropout2=0.3807450679193153,
                           model__dropout3=0.6916033873523364, 
                           model__layer1=167, 
                           model__layer2=337, 
                           model__layer3=289, 
                           model__learning_rate=0.0013556548021189216,
                           model__optim=Adam,
                           model__second_dense=True,
                           validation_split=0.16231298330980526, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [51]:
model.fit(X_train_, y_train)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_6 (Masking)         (None, 33, 25)            0         
                                                                 
 lstm_6 (LSTM)               (None, 167)               128924    
                                                                 
 dropout_11 (Dropout)        (None, 167)               0         
                                                                 
 dense_11 (Dense)            (None, 337)               56616     
                                                                 
 dropout_12 (Dropout)        (None, 337)               0         
                                                                 
 dense_12 (Dense)            (None, 289)               97682     
                                                                 
 dropout_13 (Dropout)        (None, 289)              

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=229, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002007EAFDFD0>, <keras.callbacks.LambdaCallback object at 0x0000020009161910>], epochs=37, model=<function create_model at 0x0000020009692AF0>, model__activation1='tanh', model__activation2='relu', model__activation3='sigmoid', model__dropout1=0.43...ss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000201B29833D0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000201B2983400>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, model__second_dense=True, print_summary=True, validation_split=0.16231298330980526, verbose=0),
                     clip_y=120, include_settings=True, seq_length=33)

In [52]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.756,RMSE=-20.492
Finished: 2022-11-03 10:42:57.002708


# LSTM-LSTM-Dense (TODO)

## Linear RUL

Score: 0.4617027829919557  
Test: 0.
```
('basemodel__batch_size', 107),
('basemodel__epochs', 24),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'tanh'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__dropout1', 0.7024252271635242),
('basemodel__model__dropout2', 0.16380096029242686),
('basemodel__model__dropout3', 0.30428197114787425),
('basemodel__model__layer1', 16),
('basemodel__model__layer2', 161),
('basemodel__model__layer3', 291),
('basemodel__model__learning_rate', 0.002733349824393456),
('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
('basemodel__model__second_dense', False),
('basemodel__validation_split', 0.10395419197584942),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 31)
```






In [53]:
SEQ_LENGTH=31
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=107,
                           epochs=24,
                           model__activation1='tanh',
                           model__activation2='tanh',
                           model__activation3='selu',
                           model__dropout1=0.7024252271635242, 
                           model__dropout2=0.16380096029242686,
                           model__dropout3=0.30428197114787425, 
                           model__layer1=16, 
                           model__layer2=161, 
                           model__layer3=291, 
                           model__learning_rate=0.002733349824393456,
                           model__optim=RMSprop,
                           model__second_dense=False,
                           validation_split=0.10395419197584942, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [54]:
model.fit(X_train_, y_train)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_7 (Masking)         (None, 31, 25)            0         
                                                                 
 lstm_7 (LSTM)               (None, 31, 16)            2688      
                                                                 
 dropout_14 (Dropout)        (None, 31, 16)            0         
                                                                 
 lstm_8 (LSTM)               (None, 161)               114632    
                                                                 
 dropout_15 (Dropout)        (None, 161)               0         
                                                                 
 dense_14 (Dense)            (None, 291)               47142     
                                                                 
 dropout_16 (Dropout)        (None, 291)              

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=107, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002007EAFDFD0>, <keras.callbacks.LambdaCallback object at 0x0000020009161910>], epochs=24, model=<function create_model at 0x0000020009692AF0>, model__activation1='tanh', model__activation2='tanh', model__activation3='selu', model__dropout1=0.70242...__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x0000020174E5CAF0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x0000020174E5CEE0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__second_dense=False, print_summary=True, validation_split=0.10395419197584942, verbose=0),
                     include_settings=True, seq_length=31)

In [55]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.278,RMSE=-46.342
Finished: 2022-11-03 10:47:23.442986


## Non-Linear RUL

Score: 0.6675592008245431  
Test: 0.
```
('basemodel__batch_size', 450),
('basemodel__epochs', 48),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'tanh'),
('basemodel__model__activation3', 'elu'),
('basemodel__model__dropout1', 0.8145333249976955),
('basemodel__model__dropout2', 0.4289024842480126),
('basemodel__model__dropout3', 0.1),
('basemodel__model__layer1', 496),
('basemodel__model__layer2', 168),
('basemodel__model__layer3', 254),
('basemodel__model__learning_rate', 0.003118118507191013),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__model__second_dense', False),
('basemodel__validation_split', 0.49238676803302384),
('clip_y', 120),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 42)
```

In [66]:
SEQ_LENGTH=42
CLIP=120

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=450,
                           epochs=48,     
                           model__activation1='tanh',
                           model__activation2='tanh',
                           model__activation3='elu',
                           model__dropout1=0.8145333249976955, 
                           model__dropout2=0.4289024842480126,
                           model__dropout3=0.1, 
                           model__layer1=496, 
                           model__layer2=168, 
                           model__layer3=254, 
                           model__learning_rate=0.003118118507191013,
                           model__optim=Adam,
                           model__second_dense=False,
                           validation_split=0.49238676803302384, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [67]:
model.fit(X_train_, y_train)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_11 (Masking)        (None, 42, 25)            0         
                                                                 
 lstm_14 (LSTM)              (None, 42, 496)           1035648   
                                                                 
 dropout_25 (Dropout)        (None, 42, 496)           0         
                                                                 
 lstm_15 (LSTM)              (None, 168)               446880    
                                                                 
 dropout_26 (Dropout)        (None, 168)               0         
                                                                 
 dense_22 (Dense)            (None, 254)               42926     
                                                                 
 dropout_27 (Dropout)        (None, 254)             

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=450, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002007EAFDFD0>, <keras.callbacks.LambdaCallback object at 0x0000020009161910>], epochs=48, model=<function create_model at 0x0000020009692AF0>, model__activation1='tanh', model__activation2='tanh', model__activation3='elu', model__dropout1=0.814533...s='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000201D0458E20>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000002001A3A8BB0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, model__second_dense=False, print_summary=True, validation_split=0.49238676803302384, verbose=0),
                     clip_y=120, include_settings=True, seq_length=42)

In [68]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.775,RMSE=-19.677
Finished: 2022-11-03 12:56:20.532875
